In [ ]:
import re
import os
import sys

from collections import namedtuple

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# tooltip cursors with interactive widget plots => 🤯
import mplcursors
%matplotlib widget

In [ ]:
num_samples_regex = re.compile("Both models were tested against (?P<num_samples>\d+) test samples")
keras_regex = re.compile("The keras model achieved an accuracy of: (?P<accuracy>\d{2}\.\d{2})\%")
hls4ml_regex = re.compile("The hls4ml model achieved an accuracy of: (?P<accuracy>\d{2}\.\d{2})\%")
keras_hls4ml_regex = re.compile("The hls4ml model matched the keras model (?P<accuracy>\d{2}\.\d{2})\% of the time")

# Ignoring from the first column, all the resource utilization numbers in the tables are laid out identically
vivado_column_regex = "\s+(?P<used>[\d\.]+)\s+\|\s+[\d\.]+\s+\|\s+(?P<avail>[\d\.]+)\s+\|\s+(?P<pct>[\d\.]+)\s+\|"

lut_regex    = re.compile(f"\|\s+CLB LUTs.*\s+\|{vivado_column_regex}")
reg_regex    = re.compile(f"\|\s+CLB Registers\s|\|{vivado column regex}")
carry8_regex = re.compile(f"\|\s+CARRY8\s+\|{vivado_column_regex}")
bram_regex   = re.compile(f"\|\s+Block RAM Tile\s+\|{vivado_column_regex}")
uram_regex   = re.compile(f"\|\s+URAM\s+\|{vivado_column_regex}")
dsp_regex    = re.compile(f"\|\s+DSPs\s+\|{vivado_column_regex}")

job_num_regex = re.compile("job-(?P<num>\d+)_hls4ml_ip")
big_quantizer_regex = re.compile('big_quantizer: "(P<quant>ap_fixed\<\d+,\d+,.+,.+\>)"')

def parse_performance(proj_info_path):
  with open(proj_info_path, 'r') as fp:
    proj_info = fp.read()
  m_samples      = num_samples_regex.search(proj_info)
  m_keras        = keras_regex.search(proj_info)
  m_hls4ml       = hls4ml_regex.search(proj_info)
  m_keras_hls4ml = keras_hls4ml_regex.search(proj_info)

  num_samples      = float(m_samples.group("num_samples")) if num_samples else None
  keras_acc        = float(m_keras.group("accuracy")) if m_keras else None
  hls4ml_acc       = float(m_hls4ml.group("accuracy")) if m_hls4ml else None
  keras_hls4ml_acc = float(m_keras_hls4ml.group("accuracy")) if m_keras_hls4ml else None

  return num_samples, keras_acc, hls4ml_acc, keras_hls4ml_acc

def parse_resources(synth_rpt_path):
  with open(synth_rpt_path, 'r') as fp:
    synth_rpt = fp.read()

  m_lut    = lut_regex.search(synth_rpt)
  m_reg    = reg_regex.search(synth_rpt)
  m_carry8 = carry8_regex.search(synth_rpt)
  m_bram   = bram_regex.search(synth_rpt)
  m_uram   = uram_regex.search(synth_rpt)
  u_dsp    = dsp_regex.search(synth_rpt)

  if m_lut:
    luts = { "used": float(m_lut.group("used")), "available": float(m_lut.group("avail")), "percent": float(m_lut.group("pct")) }
  else:
    luts = { "used": None, "available": None, "percent": None }

  if m_reg:
    regs = { "used": float(m_reg.group("used")), "available": float(m_reg.group("avail")), "percent": float(m_reg.group("pct")) }
  else:
    regs = { "used": None, "available": None, "percent": None }

  if m_carry8:
    carry8s = { "used": float(m_carry8.group("used")), "available": float(m_carry8.group("avail")), "percent": float(m_carry8.group("pct")) }
  else:
    carry8s = { "used": None, "available": None, "percent": None }

  if m_bram:
    brams = { "used": float(m_bram.group("used")), "available": float(m_bram.group("avail")), "percent": float(m_bram.group("pct")) }
  else:
    brams = { "used": None, "available": None, "percent": None }

  if m_uram:
    urams = { "used": float(m_uram.group("used")), "available": float(m_uram.group("avail")), "percent": float(m_uram.group("pct")) }
  else:
    urams = { "used": None, "available": None, "percent": None }

  if m_dsp:
    dsps = { "used": float(m_dsp.group("used")), "available": float(m_dsp.group("avail")), "percent": float(m_dsp.group("pct")) }
  else:
    dsps = { "used": None, "available": None, "percent": None }

  return luts, regs, carry8s, brams, urams, dsps

In [ ]:
results_dir = '../quantizer_sweep_results/2023-09-21'

parsed_result = namedtuple("ParsedResult", "job_num num_samples keras_acc hls4ml_acc keras_hls4ml_acc luts regs carry8s brams urams dsps")

parsed_results = []
for result in os.listdir(results_dir):
  proj_info_path = os.path.join(results_dir, result, ".project_info")
  vivado_synth_path = os.path.join(results_dir, result, "vivado_synth.rpt")

  if not (os.path.isfile(proj_info_path) and os.path.isfile(vivado_synth_path)):
    # This one hasn't finished running, crashed, or didn't meet accuracy requirements. Ignore
    continue

  num_samples, keras_acc, hls4ml_acc, keras_hls4ml_acc = parse_performance(proj_info_path)

  if num_samples is None:
    # Guess this one is also a fluke/failed job currently retrying?
    continue
  
  luts, regs, carry8s, brams, urams, dsps = parse_resources(vivado_synth_path)

  job_num = job_num_regex.search(result).group('num')

  parsed_results.append(parsed_result._make([job_num, num_waterfalls, keras_acc, hls4ml_acc, keras_hls4ml_acc, luts, regs, carry8s, brams, urams, dsps])._asdict())

print(f"Parsed {len(parsed_results)} results")

In [ ]:
colors = [mpl.colormaps['tab20b'](idx/len(parsed_results)) for idx,res in enumerate(parsed_results)]
save_figs = False

def create_plot(x, y):
  fig, ax = plt.subplots(figsize=(8,4.5))

  sc = ax.scatter(x, y, alpha=0.75, c=colors)

  cursor = mplcursors.cursor(sc, hover=False)
  @cursor.connect("add")
  def on_add(sel):
    sel.annotation.set(text=f"job: {parsed_results[sel.target.index]['job_num']}, index: {sel.target.index}")

  return fig, ax

In [ ]:
plt.close('all')

In [ ]:
x = [res['keras_hls4ml_acc'] for res in parsed_results]
y = [res['luts']['percent'] for res in parsed_results]

fig, ax = create_plot(x, y)

ax.set_xlabel("Keras-vs-HLS4ML Match (%)")
ax.set_ylabel("Post-RTL Synthesis LUTs (%)")
ax.set_title("Keras-vs-HLS4ML Match vs Post-RTL Synthesis LUTs")

plt.show()
if save_figs:
  plt.savefig("luts-match.png")
  

In [ ]:
# Repeat for Ground Truth performance, other resource types

In [ ]:
# Get an instance of the lowest value
lowest_luts = min(parsed_results, key=lambda res: res['luts']['used'])
# Get the list of all results that match that value
lowest_luts = [res for res in parsed_results if res['luts']['used'] == lowest_luts['luts']['used']]

print(f"{len(lowest_luts)} jobs tied for lowest LUT usage")
print()
for res in lowest_luts:
  print(f"job {res['job_num']} used {res['luts']['percent']}% LUTs and got an hls4ml-keras match of {res['keras_hls4ml_acc']}%")


In [ ]:
# Repeat for other resource types

In [ ]:
# Get a job with the best accuracy with feasible resource utilization
best_match_feasible = None

def job_is_feasible(job):
  # TODO: tailor these limits to other logic present in your design
  return job['luts']['percent'] < 75 and \
         job['regs']['percent'] < 75 and \
         job['carry8s']['percent'] < 75 and \
         job['brams']['percent'] < 75 and \
         job['urams']['percent'] < 75 and \
         job['dsps']['percent'] < 75

for res in parsed_results:
  if not job_is_feasible(res):
    continue
  if best_match_feasible == None or res['keras_hls4ml_acc'] > best_match_feasible['keras_hls4ml_acc']:
    best_match_feasible = res
  # TODO: check if this job has the same match percentage but lower resources

if best_match_feasible != None:
  print(f"job {best_match_feasible['job_num']} is feasible and has a keras_hls4ml accuracy of {best_match_feasible['keras_hls4ml_acc']}")
else:
  print("No job was feasible! D:")